In [47]:
import numpy as np
import chainer
import matplotlib.pyplot as plt
%matplotlib inline
import yaml
import quippy

In [2]:
from modules.data import AtomicStructureDataset
from modules.model import HDNNP, SingleNNP
from modules.util import DictAsAttributes, HyperParameter

/Users/jas/anaconda/envs/python2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


### スーパーセルについての計算チェック

In [3]:
config = DictAsAttributes({'preconditioning': 'none'})
with open('hyperparameter.yaml') as f:
    hp_dict = yaml.load(f)
    dataset_hp = DictAsAttributes(hp_dict['dataset'])
    model_hp = HyperParameter(hp_dict['model'], random=None)
hp = config + dataset_hp + model_hp[0]

In [4]:
datasets = []
for i in range(3):
    dataset = AtomicStructureDataset(hp)
    dataset.load_xyz('GaN/debug_supercell/type{}/structure.xyz'.format(i+1))
    datasets.append(dataset)

In [28]:
# input　sizeをNoneにしているため、init段階では最初のレイヤーの重みが初期化されていない(None)
# なぜかわからないが、一度hdnnp.predictで通した後sync_param_withをするとmastersのレイヤーの重みが初期化されるので以下のような変なことをしている
masters = chainer.ChainList(*[SingleNNP(hp, element) for element in ['Ga', 'N']])
hdnnps = []
for i in range(3):
    hdnnp = HDNNP(hp, datasets[i].composition)
    hdnnps.append(hdnnp)
for i in range(3):
    hdnnps[i].predict(datasets[i].input, datasets[i].dinput)
    hdnnps[i].sync_param_with(masters)

In [36]:
E,F = [],[]
for i in range(3):
    energy,force = hdnnps[i].predict(datasets[i].input, datasets[i].dinput)
    E.append(energy.data)
    F.append(force.data)
print E[0]/E[0], E[1]/E[0], E[2]/E[0]
print F[0][:,0:4]
print F[1][:,0:4]
print F[2][:,0:4]

[[ 1.]] [[ 7.99999714]] [[ 26.99999809]]
[[[  6.14880014e-09  -1.11391536e-08   9.17771086e-03]
  [  3.75754095e-09   2.72514278e-08   9.17770341e-03]
  [ -2.70697775e-09   3.36539756e-08   9.17570386e-03]
  [ -5.97456129e-09  -3.23341212e-08   9.17576626e-03]]]
[[[ -1.54420263e-08  -2.11202291e-08   9.17771459e-03]
  [ -4.63041943e-08   2.01289367e-08   9.17769130e-03]
  [ -9.34010114e-09  -2.98945011e-08   9.17570014e-03]
  [ -2.98565439e-08   1.49027315e-08   9.17569734e-03]]]
[[[ -8.29459168e-09  -3.02097760e-08   9.17768292e-03]
  [ -6.98491931e-09   2.65426934e-08   9.17773508e-03]
  [  3.89991328e-09  -2.63389666e-08   9.17568989e-03]
  [ -2.01990442e-08   1.03609636e-08   9.17568896e-03]]]


あるprimitiveセルと、2x2x2のsupercell、3x3x3のsupercellについて計算した。  
エネルギーはちゃんと8倍、27倍になっており、axis=2、つまりz方向の力も一致しているが、  
x,y方向への力がおかしい（そのまま拡張子しているだけなので、力が一致していないとおかしい）

In [39]:
sf1 = np.load('GaN/debug_supercell/type1/Symmetry_Function.npz')
sf2 = np.load('GaN/debug_supercell/type2/Symmetry_Function.npz')
sf3 = np.load('GaN/debug_supercell/type3/Symmetry_Function.npz')

In [117]:
# あるsample、ある原子、あるG4のmixのSF
print sf1['dG/type4/5.0/0.1/-1/2'][0,0,2,:,:]

[[  1.79477411e-09   1.31228717e-09   1.00545803e-04]
 [  9.89171633e-10   1.50497570e-09  -4.46350779e-04]
 [  4.49160709e-11   1.18586474e-10  -1.47126503e-02]
 [  1.21343477e-10   4.80573137e-10   1.50072305e-02]]


In [116]:
a = np.zeros((4,3))
for i in range(4):
    a[i] = sf2['dG/type4/5.0/0.1/-1/2'][0,0,2,:,:][range(i,32,4)].sum(axis=0)
print a

[[ -2.20157825e-09   3.16425641e-09   1.00545978e-04]
 [ -9.04983977e-10   1.35580414e-09  -4.46340942e-04]
 [ -1.06297193e-11   1.17879040e-10  -1.47126587e-02]
 [ -2.59264255e-09  -2.58646793e-09   1.50072305e-02]]


In [113]:
key = 'dG/type2/5.0/0.1/2.0'
print sf1[key][0,0,1,:,:]
a = np.zeros((4,3))
for i in range(4):
    a[i] = sf2[key][0,0,1,:,:][range(i,32,4)].sum(axis=0)
print a

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -1.41261011e-01]
 [ -2.67209543e-09   4.04270395e-09   1.35732263e-01]]
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00  -1.41261011e-01]
 [  2.16095941e-09  -3.56203600e-09   1.35732278e-01]]


In [52]:
atoms = quippy.AtomsList('GaN/debug_supercell/GaN.xyz')
for at in atoms:
    at.set_cutoff(5.0)
    at.calc_connect()
    neighb_list = at.connect.get_neighbours(1)[0] - 1
    print len(neighb_list)
    print [[(neighb_list == i+j*4).sum() for j in range(at.n / 4)] for i in range(4)]

47
[[6], [12], [14], [15]]
47
[[0, 0, 2, 0, 2, 0, 2, 0], [1, 1, 1, 1, 1, 1, 3, 3], [1, 1, 2, 2, 2, 2, 2, 2], [3, 1, 3, 1, 3, 1, 3, 0]]
47
[[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1], [1, 1, 0, 2, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0]]


In [140]:
def calc_G4(at, dtype, Rc=5.0, eta=0.1, lambda_=-1, zeta=2):
    natom = at.n
    at.set_cutoff(Rc)
    at.calc_connect()
    #for i in range(natom):
    for i in range(1):
        n_neighb = at.n_neighbours(i+1)
        if n_neighb == 0:
            continue
        r = np.zeros((n_neighb, 3), dtype=dtype)
        for l, neighb in enumerate(at.connect[i+1]):
            #print neighb.j%4, neighb.diff, neighb.shift
            r[l] = neighb.diff
        R = np.linalg.norm(r, axis=1)
        tanh = np.tanh(1 - R/Rc)
        dR = r / R[:, None]
        cos = dR.dot(dR.T)
        dcos = - r[:, None, :]/R[:, None, None]**2 * cos[:, :, None] + r[None, :, :]/(R[:, None, None] * R[None, :, None])
        ang = 1. + lambda_ * cos
        rad1 = np.exp(-eta * R**2) * tanh**3
        rad2 = np.exp(-eta * R**2) * tanh**2 * (-2.*eta*R*tanh + 3./Rc*(tanh**2 - 1.0))
        ang[np.eye(len(R), dtype=bool)] = 0
        g = 2.**(1-zeta) * ang**zeta * rad1[:, None] * rad1[None, :]
        dg_radial_part = 2.**(1-zeta) * ang[:, :, None]**zeta * rad2[:, None, None] * rad1[None, :, None] * dR[:, None, :]
        dg_angular_part = zeta * lambda_ * 2.**(1-zeta) * ang[:, :, None]**(zeta-1) * rad1[:, None, None] * rad1[None, :, None] * dcos
        dg = dg_radial_part + dg_angular_part
        
        neighbour_list = at.connect.get_neighbours(i+1)[0] - 1
        dG = np.zeros((natom,3))
        for j in range(natom):
            dG[j] = dg.take(np.where(neighbour_list == j)[0],0).sum(axis=(0,1))
        return dG

In [144]:
dG1 = calc_G4(atoms[0], dtype=np.float32)
dG2 = calc_G4(atoms[1], dtype=np.float32)
print dG1
dG2_sum = np.zeros((4,3))
for i in range(4):
    dG2_sum[i] = dG2[range(i,32,4)].sum(axis=0)
print dG2_sum

[[  1.12546417e-09   6.28205044e-10   1.00545891e-04]
 [ -7.71238851e-10   6.18051388e-09  -4.46346734e-04]
 [ -8.62785399e-11   1.97350092e-09  -5.59070706e-02]
 [  5.03100051e-09   1.31063267e-08   5.65038174e-02]]
[[ -9.39348932e-10   2.67435165e-09   1.00546094e-04]
 [ -1.02091807e-09  -9.02279379e-10  -4.46341990e-04]
 [ -9.45421566e-10   4.37966248e-10  -5.59070448e-02]
 [  7.22030448e-09   5.00605268e-09   5.65037640e-02]]


In [145]:
dG1 = calc_G4(atoms[0], dtype=np.float64)
dG2 = calc_G4(atoms[1], dtype=np.float64)
print dG1
dG2_sum = np.zeros((4,3))
for i in range(4):
    dG2_sum[i] = dG2[range(i,32,4)].sum(axis=0)
print dG2_sum

[[ -2.25906057e-11   6.48932981e-11   1.00545738e-04]
 [  2.57497791e-12  -4.12516240e-11  -4.46346644e-04]
 [  1.81630610e-11  -4.66442943e-11  -5.59070487e-02]
 [  1.40634883e-10  -2.39906140e-10   5.65037597e-02]]
[[ -2.25906058e-11   6.48932862e-11   1.00545738e-04]
 [  2.57497450e-12  -4.12516190e-11  -4.46346644e-04]
 [  1.81630591e-11  -4.66442861e-11  -5.59070487e-02]
 [  1.40634904e-10  -2.39906105e-10   5.65037597e-02]]
